In [3]:
import mysql.connector

mydb_conn = mysql.connector.connect(
    host = "mysql-library-database-sql-database-project-1.e.aivencloud.com",
    user = "avnadmin",
    password = "AVNS_UOValGtpsHEmocnN6I5",
    port=28505,
    database='library'
)

print(mydb_conn)

In [4]:
cursor = mydb_conn.cursor()
cursor

In [8]:
cursor.execute("USE library")


result = cursor.fetchall()
result

[]

In [10]:
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("Tables in database:", tables)

Tables in database: [('Books',), ('Loans',), ('Patrons',)]


In [12]:
# /-- 1- find the number of availalbe copies of the book (Dracula)

cursor.execute("""
SELECT COUNT(*) as avail_copies_of_Dracula FROM Books 
WHERE Title = 'Dracula' AND BookID NOT IN (
SELECT BookID FROM Loans WHERE ReturnedDate IS NULL
);
""")

result = cursor.fetchall()
result

[(2,)]

In [13]:
# /-- 2- check total copies of the book (answer 1)
cursor.execute("""
SELECT COUNT(*) as total_copies FROM Books
WHERE Title = 'Dracula';
""")

result = cursor.fetchall()
result

[(3,)]

In [14]:
# /-- 2- check total copies of the book (answer 2)
cursor.execute("""
SELECT * FROM Books
WHERE Title = 'Dracula';
""")

result = cursor.fetchall()
result

[(12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (60, 'Dracula', 'Bram Stoker', 1897, 4274716715),
 (73, 'Dracula', 'Bram Stoker', 1897, 8730298424)]

In [15]:
# /-- 3- current total loans of the book  (answer 1)
cursor.execute("""
SELECT COUNT(*) AS total_loans_of_Dracula FROM Loans l
JOIN Books b ON l.BookID = b.BookID
WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[(1,)]

In [16]:
# /-- 3- current total loans of the book  (answer 2)
cursor.execute("""
SELECT * FROM Loans l
JOIN Books b ON l.BookID = b.BookID
WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[(1999,
  73,
  59,
  '2020-06-29',
  '2020-07-13',
  None,
  73,
  'Dracula',
  'Bram Stoker',
  1897,
  8730298424)]

In [18]:
# /-- 4- total available books of dracula */ (answer 1)
cursor.execute("""
SELECT COUNT(*) AS avaialble_dracula_books FROM Books
WHERE Title = 'Dracula' AND BookID NOT IN (
SELECT BookID FROM Loans WHERE ReturnedDate IS NULL
);
""")

result = cursor.fetchall()
result

[(2,)]

In [19]:
# /-- 4- total available books of dracula */ (answer 2)
cursor.execute("""
SELECT * FROM Books
WHERE Title = 'Dracula' AND BookID NOT IN (
SELECT BookID FROM Loans WHERE ReturnedDate IS NULL
);
""")

result = cursor.fetchall()
result

[(12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (60, 'Dracula', 'Bram Stoker', 1897, 4274716715)]

In [20]:
# /-- 5- Add new books to the library
cursor.execute("""              
INSERT INTO Books (BookID, Title, Author, Published, Barcode) VALUES
(2001, 'Don Quixote', 'Miguel de Cervantes', 1860, 9780141439846),
(2002, 'Alice Adventures in Wonderland', 'Lewis Carroll', 1900, 9780199564095),
(2003, 'Treasure Island', '	Robert Louis Stevenson', 1897, 9780141045227),
(2004, 'Frankenstein', 'Mary Shelley', 1818, 9780486282114),
(2005, 'The Shining', 'Stephen King', 1977, 9780307743657);
""")

result = cursor.fetchall()
result

[]

In [21]:
# /-- 5- Add new books to the library (showing the list with the new added books)
cursor.execute("SELECT * FROM Books")

result = cursor.fetchall()
result

[(1,
  'Walden, and On The Duty Of Civil Disobedience',
  'Henry David Thoreau',
  1849,
  3100471153),
 (2, 'The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 4589408958),
 (3,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  2411676280),
 (4, 'The Adventures of Tom Sawyer', 'Mark Twain', 1876, 7813216705),
 (5, 'David Copperfield', 'Charles Dickens', 1850, 8821265677),
 (6, 'A Modest Proposal', 'Jonathan Swift', 1729, 9675136364),
 (7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 (8,
  'The Importance of Being Earnest: A Trivial Comedy for Serious People',
  'Oscar Wilde',
  1895,
  4733732465),
 (9, "Uncle Tom's Cabin", 'Harriet Beecher Stowe', 1852, 1874764443),
 (10, 'The Count of Monte Cristo', 'Alexandre Dumas', 1884, 1799238621),
 (11, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 2855934983),
 (12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (13, 'Oliver Twist', 'Charles Dickens', 1839, 3071366111),
 (14,
  'The Awakening, an

In [22]:
#/ - 6- Check out Books: books(4043822646, 2855934983) whose patron_email(jvaan@wisdompets.com), loandate=2020-08-25, duedate=2020-09-08, loanid=by_your_choice*/

cursor.execute("""
SELECT BookID, PatronID, '2020-08-25', '2020-09-08' FROM Books, Patrons
WHERE Email = 'jvaan@wisdompets.com' AND Barcode IN (4043822646, 2855934983);
""")

result = cursor.fetchall()
result

[(93, 50, '2020-08-25', '2020-09-08'), (11, 50, '2020-08-25', '2020-09-08')]

In [23]:
#/ -- 7- Check books for Due back
cursor.execute("""
SELECT l.LoanID, b.Title, p.FirstName, p.LastName, p.Email, l.LoanDate, l.DueDate FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[(1800,
  'Three Men in a Boat (To Say Nothing of the Dog)',
  'Clem',
  'Durdy',
  'cdurdy2p@hplussport.com',
  '2020-02-08',
  '2020-02-22'),
 (1864,
  'Jane Eyre: An Autobiography',
  'Maggi',
  'Domney',
  'mdomney0@wisdompets.com',
  '2020-03-24',
  '2020-04-07'),
 (1911,
  'Jane Eyre: An Autobiography',
  'Christyna',
  'McGilroy',
  'cmcgilroy11@landonhotel.com',
  '2020-04-24',
  '2020-05-08'),
 (1948,
  'Sense and Sensibility',
  'Arlen',
  "O'Fihillie",
  'aofihillie1f@wisdompets.com',
  '2020-05-17',
  '2020-05-31'),
 (1983,
  'David Copperfield',
  'Gus',
  'Holyland',
  'gholyland2f@red30design.com',
  '2020-06-17',
  '2020-07-01'),
 (1984,
  'An Occurrence at Owl Creek Bridge',
  'Viola',
  'McLarnon',
  'vmclarnon24@orangevalleycaa.org',
  '2020-06-18',
  '2020-07-02'),
 (1985,
  "Grimms' Fairy Tales",
  'Robbert',
  'Walkowski',
  'rwalkowski1i@kinetecoinc.com',
  '2020-06-18',
  '2020-07-02'),
 (1986,
  'Walden, and On The Duty Of Civil Disobedience',
  'Chance',
  'To

In [24]:
#/ -- 8- generate a report of books due back on July 13, 2020 */

cursor.execute("""
SELECT l.LoanID, b.Title, b.BookID, l.LoanDate, l.DueDate FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.DueDate = '2020-07-13' 
AND l.ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[(1996, 'The Count of Monte Cristo', 10, '2020-06-29', '2020-07-13'),
 (1997, 'The Prophet', 155, '2020-06-29', '2020-07-13'),
 (1998, 'Second Treatise of Government', 169, '2020-06-29', '2020-07-13'),
 (1999, 'Dracula', 73, '2020-06-29', '2020-07-13')]

In [25]:
#/ 9-generate a report of books due back on July 13, 2020 with patron contact information 

cursor.execute("""
SELECT l.LoanID, b.Title, b.BookID, p.FirstName, p.LastName, p.Email, l.LoanDate, l.DueDate FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.DueDate = '2020-07-13' AND l.ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[(1996,
  'The Count of Monte Cristo',
  10,
  'Kala',
  'Simonian',
  'ksimonian2q@landonhotel.com',
  '2020-06-29',
  '2020-07-13'),
 (1997,
  'The Prophet',
  155,
  'Lydie',
  'Awmack',
  'lawmack1a@samoca.org',
  '2020-06-29',
  '2020-07-13'),
 (1998,
  'Second Treatise of Government',
  169,
  'Anthe',
  'Dinjes',
  'adinjesh@samoca.org',
  '2020-06-29',
  '2020-07-13'),
 (1999,
  'Dracula',
  73,
  'Cornelia',
  'Koppe',
  'ckoppe1m@landonhotel.com',
  '2020-06-29',
  '2020-07-13')]

In [27]:
#/ -- 10- Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05) code1

cursor.execute("""
UPDATE Loans 
SET ReturnedDate = '2020-07-05'
WHERE BookID = (SELECT BookID FROM Books WHERE Barcode = 6435968624)
AND ReturnedDate IS NULL;
""")

result = cursor.fetchall()
result

[]

In [28]:
#/ -- 10- Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05) code2
cursor.execute("""
SELECT * FROM Loans 
WHERE BookID = (SELECT BookID FROM Books WHERE Barcode = 6435968624);
""")

result = cursor.fetchall()
result

[(133, 105, 67, '2016-09-07', '2016-09-21', '2016-09-21'),
 (152, 105, 48, '2016-09-22', '2016-10-06', '2016-10-06'),
 (462, 105, 64, '2017-04-22', '2017-05-06', '2017-05-06'),
 (567, 105, 5, '2017-07-15', '2017-07-29', '2017-07-29'),
 (728, 105, 77, '2017-12-04', '2017-12-18', '2017-12-18'),
 (978, 105, 93, '2018-06-20', '2018-07-04', '2018-07-04'),
 (1125, 105, 17, '2018-09-29', '2018-10-13', '2018-10-13'),
 (1515, 105, 80, '2019-07-17', '2019-07-31', '2019-07-31'),
 (1552, 105, 72, '2019-08-11', '2019-08-25', '2019-08-25'),
 (1712, 105, 75, '2019-12-06', '2019-12-20', '2019-12-20'),
 (1951, 105, 52, '2020-05-20', '2020-06-03', '2020-06-03'),
 (1991, 105, 65, '2020-06-25', '2020-07-09', '2020-07-05')]

In [29]:
#/-- 11- Encourage Patrons to check out books 
#/-- a code to check patrons that didn't check out any books for the last 3 months 
cursor.execute("""
SELECT FirstName, LastName, Email FROM Patrons
WHERE PatronID NOT IN (
SELECT DISTINCT PatronID FROM Loans
WHERE LoanDate >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH)
);
""")

result = cursor.fetchall()
result

[('Maggi', 'Domney', 'mdomney0@wisdompets.com'),
 ('Javier', 'Dawks', 'jdawks1@red30design.com'),
 ('Aleen', 'Fasey', 'afasey2@kinetecoinc.com'),
 ('Taylor', 'Jenkins', 'tjenkins@rouxacademy.org'),
 ('Imogen', 'Kabsch', 'ikabsch@landonhotel.com'),
 ('Don', 'Weingarten', 'dweingarten@kinetecoinc.com'),
 ('Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org'),
 ('Steffie', 'Kleis', 'skleis7@wisdompets.com'),
 ('Carilyn', 'Calver', 'ccalver8@samoca.org'),
 ('Barbara-anne', 'Sweet', 'bsweet9@samoca.org'),
 ('Faina', 'Nelles', 'fnellesa@red30design.com'),
 ('Orton', 'Stavers', 'ostaversb@orangevalleycaa.org'),
 ('Jobina', 'McKern', 'jmckernc@rouxacademy.org'),
 ('Paco', 'Yarranton', 'pyarrantond@rouxacademy.org'),
 ('Chance', 'Todeo', 'ctodeoe@wisdompets.com'),
 ('Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com'),
 ('Lani', 'Over', 'loverg@wisdompets.com'),
 ('Anthe', 'Dinjes', 'adinjesh@samoca.org'),
 ('Amby', 'Harber', 'aharberi@hplussport.com'),
 ('Jacinta', 'Vallis', 'jvallisj@kinetecoinc.com

In [30]:
#/-- 11- Encourage Patrons to check out books 
#/-- code to check the 10 books with the most numbers of loans that can be reccommended to the patrons
cursor.execute("""
SELECT b.Title, MIN(b.Author) AS Author, COUNT(l.BookID) AS loans_count FROM Loans l
JOIN Books b ON l.BookID = b.BookID
GROUP BY b.Title
ORDER BY loans_count DESC
LIMIT 10;
""")

result = cursor.fetchall()
result

[('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  53),
 ('The Awakening, and Selected Short Stories', 'Kate Chopin', 43),
 ('The War of the Worlds', 'H. G. Wells', 43),
 ('The Scarlet Letter', 'Nathaniel Hawthorne', 42),
 ('The Picture of Dorian Gray', 'Oscar Wilde', 39),
 ("Alice's Adventures in Wonderland", 'Lewis Carroll', 38),
 ('A Tale of Two Cities', 'Charles Dickens', 37),
 ('The Turn of the Screw', 'Henry James', 37),
 ('Jane Eyre: An Autobiography', 'Charlotte Brontë', 37),
 ('The Mysterious Affair at Styles', 'Agatha Christie', 36)]

In [31]:
#/---- 12- generate a report of showing 10 patrons who have checked out the fewest books.                          */

cursor.execute("""
SELECT p.PatronID, p.FirstName, p.LastName, p.Email, COUNT(l.LoanID) AS total_checkouts_by_patrons FROM Patrons p
LEFT JOIN Loans l ON p.PatronID = l.PatronID
GROUP BY p.PatronID, p.FirstName, p.LastName, p.Email
ORDER BY total_checkouts_by_patrons ASC
LIMIT 10;
""")

result = cursor.fetchall()
result

[(83, 'Caril', 'Matejic', 'cmatejic2a@orangevalleycaa.org', 9),
 (69, 'Alaster', 'Ruggles', 'aruggles1w@red30design.com', 9),
 (73, 'Merle', 'Sukbhans', 'msukbhans20@orangevalleycaa.org', 11),
 (64, 'Norby', 'Sleight', 'nsleight1r@kinetecoinc.com', 12),
 (94, 'Curr', 'Fittall', 'cfittall2l@orangevalleycaa.org', 12),
 (50, 'Jack', 'Vaan', 'jvaan@wisdompets.com', 13),
 (12, 'Orton', 'Stavers', 'ostaversb@orangevalleycaa.org', 14),
 (16, 'Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com', 14),
 (7, 'Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org', 14),
 (3, 'Aleen', 'Fasey', 'afasey2@kinetecoinc.com', 14)]

In [32]:
#/ 13- Find books to feature for an event  create a list of books from 1890s that are currently available

cursor.execute("""
SELECT BookID, Title, Author, Published, Barcode FROM Books
WHERE Published BETWEEN 1890 AND 1899 AND BookID NOT IN (
SELECT BookID FROM Loans WHERE ReturnedDate IS NULL
);
""")

result = cursor.fetchall()
result

[(3,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  2411676280),
 (7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 (8,
  'The Importance of Being Earnest: A Trivial Comedy for Serious People',
  'Oscar Wilde',
  1895,
  4733732465),
 (11, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 2855934983),
 (12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (14,
  'The Awakening, and Selected Short Stories',
  'Kate Chopin',
  1899,
  3103139173),
 (15, 'The Time Machine', 'H. G. Wells', 1895, 9610930110),
 (17, 'Les Misérables', 'Victor Hugo', 1895, 8252831254),
 (18, 'The Jungle Book', 'Rudyard Kipling', 1894, 3655763446),
 (19, 'Heart of Darkness', 'Joseph Conrad', 1899, 9334362486),
 (26,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  6117891368),
 (28, 'Heart of Darkness', 'Joseph Conrad', 1899, 2229312049),
 (43, 'The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 7476179474),
 (54, 'The Yellow Wallpaper', 

In [33]:
#/ 14- create a report to show how many books were published each year.
cursor.execute("""
SELECT Published, COUNT(BookID) as total_books_published FROM Books 
GROUP BY Published
ORDER BY Published ASC;
""")

result = cursor.fetchall()
result

[(1375, 2),
 (1532, 3),
 (1605, 2),
 (1670, 2),
 (1689, 2),
 (1726, 3),
 (1729, 3),
 (1759, 3),
 (1791, 3),
 (1811, 3),
 (1813, 3),
 (1815, 3),
 (1818, 4),
 (1839, 3),
 (1847, 6),
 (1849, 3),
 (1850, 6),
 (1851, 3),
 (1852, 3),
 (1854, 3),
 (1858, 3),
 (1859, 6),
 (1860, 1),
 (1861, 3),
 (1865, 3),
 (1869, 6),
 (1876, 3),
 (1878, 3),
 (1879, 3),
 (1880, 3),
 (1883, 3),
 (1884, 6),
 (1886, 3),
 (1887, 3),
 (1889, 3),
 (1890, 9),
 (1892, 6),
 (1894, 3),
 (1895, 9),
 (1896, 3),
 (1897, 4),
 (1898, 6),
 (1899, 5),
 (1900, 4),
 (1901, 3),
 (1902, 2),
 (1905, 3),
 (1906, 2),
 (1908, 3),
 (1911, 6),
 (1913, 2),
 (1914, 3),
 (1915, 3),
 (1920, 3),
 (1921, 2),
 (1922, 5),
 (1923, 3),
 (1977, 1)]

In [34]:
#- 15- create a report to show 5 most popular Books to check out
cursor.execute("""
SELECT b.BookID, b.Title, b.Author, COUNT(l.LoanID) AS loans_count FROM Books b
JOIN Loans l ON b.BookID = l.BookID
GROUP BY b.BookID, b.Title, b.Author
ORDER BY loans_count DESC
LIMIT 5;
""")

result = cursor.fetchall()
result

[(112, 'The War of the Worlds', 'H. G. Wells', 20),
 (153,
  'Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  20),
 (41, 'The Scarlet Letter', 'Nathaniel Hawthorne', 18),
 (180,
  'Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  17),
 (126, 'The Awakening, and Selected Short Stories', 'Kate Chopin', 17)]

Git hub link for the project:

https://github.com/nashwa-mahmoud93/library_database_project